In [1]:
import json
import sqlalchemy

with open('config.json') as json_data_file:
    data = json.load(json_data_file)

user = data['postgresql']['user']
password = data['postgresql']['password']
host = data['postgresql']['host']
db = data['postgresql']['db'] 
connection_url = 'postgresql://{user}:{password}@{host}:55553/{db}'.format(user=user, password=password, host=host, db=db)

In [ ]:
engine = sqlalchemy.create_engine(connection_url)
connection = engine.connect()
result = connection.execute('SELECT * FROM employees ORDER BY emp_no LIMIT 10')
for row in result:
    print(row)

In [ ]:
print('Fetched Results')